Automatically Scoring Student Responses

Avi Dixit and Elizabeth McBride

<b>Introduction </b> Notebook to upload the pre and post test data into pandas dataframes and apply classification algorithms to it.


In [1]:
#imports - Consolidated imports for all functions used (or will eventually be used) by the notebook

import nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import re
import math
import string
from collections import Counter
from __future__ import division

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn import svm
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

from nltk import word_tokenize
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer 
from nltk.stem import SnowballStemmer

In [2]:
#read_file is a helper function to get the '|' delimited CSV into a data frame
def read_file(filename):
    #get the file
    df = pd.read_csv(filename)
    
    #Force KIScore to int, otherwise reverts to float. Same for Answer. Forcing NaN to unicode
    df['KIScore'] = df['KIScore'].astype(np.int32)
    df['Answer'] = df['Answer'].astype('U')
    # Filters if needed later on
    #filtered_data = df["Answer"].notnull()
    #filtered_data = df[df["KIScore"] != 1 & df['Answer'].notnull() & df["KIScore"].notnull()]
    #df_narrative = df[filtered_data]
    return df

#reads in the training data into a panda - Steve 
#(code based on ANLP Notebook Intro to Pandas by Marti Hearst and Andrea Gagliano)
def read_training_data(filename):
    df_narrative = read_file(filename)
    #print the report on category breakdown, might need these counts later
    #print("Creating training data... category breakdown:")
    #sorted_product_counts = df_narrative.Category.value_counts(ascending=True)
    #print(sorted_product_counts)
    #sorted_product_counts.plot(kind='barh', figsize=(8,6), title="Categories");
    return df_narrative

Seperate the data into training and dev data.

In [3]:
#breaks the panda into a training set and a dev set - Currently only genereates dev and test data
#Modify the function later to keep some data as test data as well

def get_train_and_dev_sets(full_data, percent_dev):
    #randomize the indices
    random_index = np.random.permutation(full_data.index)
    full_data_shuffled = full_data.ix[random_index, ['WISEID', 'Answer', 'KIScore']]
    full_data_shuffled.reset_index(drop=True, inplace=True)
    
    #break down the counts for the shuffled data
    rows, columns = full_data_shuffled.shape
    train_size = round(rows*(1 - percent_dev))
    dev_size   = round(rows*percent_dev)
    
    #separate the training data from the development data
    train_data = full_data_shuffled.loc[:train_size]
    dev_data = full_data_shuffled.loc[train_size:dev_size+train_size].reset_index(drop=True)

    return train_data, dev_data 

In [4]:
#reads in the test file into a panda
def read_test_data(filename):
    #get the file
    df = read_file(filename)
    return df

In [5]:
#the code that calls the above functions - puts the data into a data frame
df = read_training_data("Question2_CSV.csv")
train_set, dev_set = get_train_and_dev_sets(df,.2)

Norvig's spell checker

In [6]:
TEXT = open('big.txt').read()

def tokens(text):
    "List all the word tokens (consecutive letters) in a text. Normalize to lowercase."
    return re.findall('[a-z]+', text.lower()) 

WORDS = tokens(TEXT)

COUNTS = Counter(WORDS)

In [7]:
def correct(word):
    "Find the best spelling correction for this word."
    # Prefer edit distance 0, then 1, then 2; otherwise default to word itself.
    candidates = (known(edits0(word)) or 
                  known(edits1(word)) or 
                  known(edits2(word)) or 
                  [word])
    return max(candidates, key=COUNTS.get)

# Show what happens in the case of ties
def correct_under_hood (word):
    candidates = (known(edits0(word)) or 
                  known(edits1(word)) or 
                  known(edits2(word)) or 
                  [word])
    return candidates

In [8]:
def known(words):
    "Return the subset of words that are actually in the dictionary."
    return {w for w in words if w in COUNTS}

def edits0(word): 
    "Return all strings that are zero edits away from word (i.e., just word itself)."
    return {word}

def edits2(word):
    "Return all strings that are two edits away from this word."
    return {e2 for e1 in edits1(word) for e2 in edits1(e1)}

In [9]:
def edits1(word):
    "Return all strings that are one edit away from this word."
    pairs      = splits(word)
    deletes    = [a+b[1:]           for (a, b) in pairs if b]
    transposes = [a+b[1]+b[0]+b[2:] for (a, b) in pairs if len(b) > 1]
    replaces   = [a+c+b[1:]         for (a, b) in pairs for c in alphabet if b]
    inserts    = [a+c+b             for (a, b) in pairs for c in alphabet]
    return set(deletes + transposes + replaces + inserts)

def splits(word):
    "Return a list of all possible (first, rest) pairs that comprise word."
    return [(word[:i], word[i:]) 
            for i in range(len(word)+1)]

alphabet = 'abcdefghijklmnopqrstuvwxyz'

In [10]:
spell_checker = lambda x : ' '.join(i for i in list(map(correct, tokens(x))))

In [11]:
train_set['Answer'] = train_set['Answer'].apply(spell_checker)

In [12]:
train_set.head()

,WISEID,Answer,KIScore
0,118562.0,it would be warmer than the outside air becaus...,3
1,136467.0,the sun gives the car a warm temperature forma...,2
2,118531.0,i think that the air in the car will be colder...,2
3,136389.0,i think that the temperature inside the car wo...,2
4,118559.0,jonathan hernandez ghyuhyuiuhi,1


In [13]:
dev_set['Answer'] = dev_set['Answer'].apply(spell_checker)

In [14]:
dev_set.head()

,WISEID,Answer,KIScore
0,139833.0,the radiation from the sun will be wrapped ins...,4
1,136351.0,i said that it would be warmer inside the car ...,2
2,118459.0,it the exactly because ti reflect the temperat...,2
3,136468.0,if its really hot outside the temperature in t...,2
4,118366.0,it was left in the sun and has not been driven...,2


<b> Strategies that need to be attempted </b>
1. Feature Selection attempted:
    1. counts of Unigrams only 
    2. ... Unigrams and Bigrams
    3. ... Unigrams, Bigrams, and Trigrams
    4. ... Bigrams and Trigrams
    5. ... 4- and 5-gram combinations
    6. The use of TF-IDF, with IDF and without
    7. Word tokens that included punctuation and numbers
    8. Word tokens with letters only, filtering punctuation or splitting on punctuation
    9. Lemmatizing using Word Net
    10. Stemming using Snowball
    11. With and without stopwords
    12. With and without lowercasing
    13. Chunking out all words that are not nouns.
    14. Stemming user Porter and Lancaster stemmers.
    15. Checking most common hypernyms of nouns in the review to categorise reviews better.
2. Classifiers used:
    1. Linear: Naive Bayes, Linear Regression, Stochastic Gradiant Descent
    2. SVC and Linear SVC (One vs One, One vs Many)
    3. K - Nearest Neighbor

In [15]:
def transform_dfs_to_arrays(train_set, dev_set):
    vec = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', stop_words="english", max_features=5000)
    arr_train_feature_sparse = vec.fit_transform(train_set["Answer"].values.astype(str))
    arr_train_feature = arr_train_feature_sparse.toarray()
    
    arr_dev_feature_sparse = vec.transform(dev_set["Answer"].values.astype(str))
    arr_dev_feature = arr_dev_feature_sparse.toarray()
    
    return arr_train_feature, arr_dev_feature


Lets start with a simple Naive Bayes classifier for Multinomial models

In [16]:
def train_NB_model(train_set, arr_train):
    nb = MultinomialNB()
    nb_model = nb.fit(arr_train, 
                      train_set.KIScore)
    return nb_model

In [17]:
arr_train, arr_dev = transform_dfs_to_arrays(train_set, dev_set)
nb_model = train_NB_model(train_set, arr_train)
nb_predictions = nb_model.predict(arr_dev)

In [18]:
accuracy_score(dev_set.KIScore, nb_predictions)

0.8018433179723502

Lets try K-nearest neighbor

In [19]:
def train_KNearest_model(train_set, arr_train):
    #Should add and experiement with more parameters and algorithms for nearest neighbor
    nb = KNeighborsClassifier(n_neighbors=5)
    nb_model = nb.fit(arr_train, 
                      train_set.KIScore)
    return nb_model

In [20]:
neigh_model = train_KNearest_model(train_set, arr_train)
ne_predictions = neigh_model.predict(arr_dev)
accuracy_score(dev_set.KIScore, ne_predictions)

0.52534562211981561

Not bad for a start, lets move onto Logistical Regression

In [21]:
def train_LR_model(train_set, arr_train):
    logreg = LogisticRegression()
    lr_model = logreg.fit(arr_train, train_set.KIScore)
    return lr_model

In [22]:
lr_model = train_LR_model(train_set, arr_train)
lr_predictions = lr_model.predict(arr_dev)

In [23]:
accuracy_score(dev_set.KIScore, lr_predictions)

0.86175115207373276

Already nearing 80s!!!! But remember, need to measure Cohen's Kappa, not percetage correct. Also start plotting confusion matrix and extract errors once the classifiers are worked out.

Lets start with the pipeline for the best features and get to feature detections using SVM. Also need to perform all the combinations mentioned before (Including preprocessing). 

In [24]:
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
        #self.snow = SnowballStemmer('english')
    
    #this code will filter punctuation from a word and rejoin it together ("they're" becomes "theyre")
    #this code was derived from original code written by Keshav Potluri, TA for INFO 206, for a class assignment
    def __preprocess(self, doc):
       filter_punc = lambda t: ''.join([x.lower() for x in t if x.isalpha()])
       words = [x for x in map(filter_punc, doc.split()) if x]
       review = ""
       for w in words:
           review = review+" "+w
       return review
    
    #Multiple attempts to select lemmas and stems from a word token (using NLTK)
    def __call__(self, doc):
        #return [self.wnl.lemmatize(t.lower()) for t in word_tokenize(doc)]
        return [self.wnl.lemmatize(t) for t in word_tokenize(self.__preprocess(doc))]
        #return ["".join([str(s.name()) for s in wn.synset(t).hypernyms()]) for t in word_tokenize(self.__preprocess(doc))]
        #return [self.snow.stem(t) for t in word_tokenize(self.__preprocess(doc))]

In [25]:
# Pipeline attempts - Best features will be decided using Grid Search. Lets just setup a baseline for now.
#from http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html
text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2), tokenizer=LemmaTokenizer(),  max_df=0.25, max_features= 15000, token_pattern=r'\b\w+\b', stop_words="english")),
                      ('tfidf', TfidfTransformer(use_idf=True,norm='l2')),
                      #('clf', SVC(C=1000000.0, gamma='auto', kernel='rbf'))])
                      ('clf', LinearSVC(C=1.0, random_state=69, penalty='l2', dual=True, tol=1e-5))])
                      #('clf', OneVsOneClassifier(LinearSVC(random_state=0)))])                    
                      #('clf', SGDClassifier(loss='hinge', alpha=1e-5, penalty='elasticnet', n_iter=50, random_state=69))])

In [26]:
pipeline_predictor = text_clf.fit(train_set["Answer"], 
                                  train_set.KIScore)
predicted = pipeline_predictor.predict(dev_set["Answer"])
accuracy_score(dev_set.KIScore, predicted)

0.82949308755760365

In [27]:
#prb = pipeline_predictor.predict_proba(dev_set["Answer"])

In [28]:
cohen_kappa_score(dev_set.KIScore, predicted)

0.56111293320214273

Woops! Not very good

Lets see which categories we are getting wrong (Don't be 2!!!!)

In [ ]:
pd.crosstab(dev_set.KIScore, predicted, 
           rownames=['True'], colnames=['Predicted'], 
            margins=True)

Predicted,1,2,3,4,All
True,,,,,
1,7,4,0,0,11
2,0,151,5,0,156
3,0,21,20,3,44
4,0,0,3,2,5
5,0,0,0,1,1
All,7,176,28,6,217


Hm, it looks like we should try and get more data. Might not be possible without mixing up student responses.

In [ ]:
parameters = {
                'vect__max_df': (.5, .25, 1.0, 5.0),
                'clf__penalty': ('l2', 'elasticnet'),
                'vect__max_features': (None, 5000, 10000, 50000),
                'tfidf__smooth_idf': (True, False),
                'tfidf__sublinear_idf': (True, False),
                'clf__penalty': ('l1', 'l2'),
                'clf__dual': (True, False),
                'clf__loss': ('hinge','squared_hinge')
 }

gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)
gs_clf = gs_clf.fit(train_set["Answer"], train_set.KIScore)
predicted_1 = gs_clf.predict(dev_set["Answer"])
accuracy_score(dev_set.KIScore, predicted_1)

print("Best score: %0.3f" % gs_clf.best_score_)
print("Best parameters set:")
best_parameters = gs_clf.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))